In [143]:
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.feature_extraction.text import CountVectorizer

 - Movies Genres Feature Enginnering 
 - Genome Tranformations 
 - Combining the data
 - Recommendation matrix

In [144]:
# Reading in the features 
movies_df1 = pd.read_csv('data/movies_cleaned.csv', low_memory = False)
genome_df = pd.read_csv('data/genome_clean.csv', low_memory = False)
movies_df1.head(2)

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995


In [145]:
genome_df.head(2)

,tagId,tag,movieId,relevance
0,1,007,1,0.02500
1,1,007,2,0.03975


In [146]:
# Creating a function to clean up genres and get the data ready for word vecotrizer
def no_genres(i):
    if i == '(no genres listed)':
        return 'no_genres'
    else:
        return i

movies_df1['genres'] = movies_df1['genres'].apply(no_genres)
movies_df1[movies_df1['genres'].str.contains('no')].head(3)

,movieId,title,genres,year
16574,83773,Away with Words (San tiao ren) (1999),no_genres,1999
16589,83829,Scorpio Rising (1964),no_genres,1964
16764,84768,Glitterbug (1994),no_genres,1994


In [147]:
movies_df1.head(3)

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995


#### Movies Genres Feature Enginnering 

In [148]:
# Create a decade feature for every movie, instead of a year

# Function to create take a floor round 
def to_decade(num):
    num_str = str(num)[:3] + '0'
    return num_str

# Assign the new feature
movies_df1['decade'] = movies_df1['year'].map(to_decade)
movies_df1.head(3)

,movieId,title,genres,year,decade
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,1990
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,1990
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,1990


In [149]:
# Creating dummy variables from decade
movies_df = pd.get_dummies(movies_df1, columns=['decade'], drop_first=False)
movies_df.head(3)

,movieId,title,genres,year,decade_1890,decade_1900,decade_1910,decade_1920,decade_1930,decade_1940,decade_1950,decade_1960,decade_1970,decade_1980,decade_1990,decade_2000,decade_2010
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,0,0,0,0,0,0,0,0,0,0,1,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,0,0,0,0,0,0,0,0,0,0,1,0,0


In [150]:
# Remove year from the dataframe, as it is not unnecessary
movies_df = movies_df.drop(columns='year',axis = 1 )

In [151]:
# Create features based each genre 

# instantiating count vectorizer
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None) 

# applying count vectorizer to yelp categories column
cat_vector = vectorizer.fit_transform(movies_df['genres']).toarray()

# merging results with original yelp data frame
cat_vector_df = pd.DataFrame(cat_vector, columns = vectorizer.get_feature_names())
cat_vector_df.shape

(27278, 32)

In [152]:
# Merging the vectorizer genres and the movie dataframes
movies_vecrt_df = pd.merge(left = movies_df, 
                            right = cat_vector_df, 
                            on=movies_df.index)

movies_vecrt_df.head(3)

,key_0,movieId,title,genres,decade_1890,decade_1900,decade_1910,decade_1920,decade_1930,decade_1940,...,imax,musical,mystery,no_genres,noir,romance,sci,thriller,war,western
0,0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,Jumanji (1995),Adventure|Children|Fantasy,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [153]:
# dropping the key_0 since it's unnecessary
movies_vecrt_df = movies_vecrt_df.drop(columns='key_0')

In [154]:
movies_vecrt_df.columns

Index(['movieId', 'title', 'genres', 'decade_1890', 'decade_1900',
       'decade_1910', 'decade_1920', 'decade_1930', 'decade_1940',
       'decade_1950', 'decade_1960', 'decade_1970', 'decade_1980',
       'decade_1990', 'decade_2000', 'decade_2010', '1970', '1990', '1991',
       '1993', '2002', '2003', '2008', '2010', '2011', '2014', 'action',
       'adventure', 'animation', 'children', 'comedy', 'crime', 'documentary',
       'drama', 'fantasy', 'fi', 'film', 'horror', 'imax', 'musical',
       'mystery', 'no_genres', 'noir', 'romance', 'sci', 'thriller', 'war',
       'western'],
      dtype='object')

In [155]:
# Rename address scf-fi movies as the category was split up

movies_vecrt_df = movies_vecrt_df.drop(columns = 'fi', axis = 1)
movies_vecrt_df = movies_vecrt_df.rename(columns = {'sci': 'scifi'})

In [156]:
# Rename address scf-fi movies as the category was split up

movies_vecrt_df = movies_vecrt_df.drop(columns = 'film', axis = 1)
movies_vecrt_df = movies_vecrt_df.rename(columns = {'noir': 'film_noir'})

In [157]:
movies_vecrt_df.head(3)

,movieId,title,genres,decade_1890,decade_1900,decade_1910,decade_1920,decade_1930,decade_1940,decade_1950,...,imax,musical,mystery,no_genres,film_noir,romance,scifi,thriller,war,western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [158]:
movies_vecrt_df['decade'] = movies_df1['decade']

In [159]:
# Exporting dataframe for data exploration
movies_vecrt_df.to_csv('data/movie_features_eda.csv',index = False)

In [160]:
movies_vecrt_df = movies_vecrt_df.drop(columns=['genres','decade'])

### Genome Tranformations 

In [161]:
genome_df.head(2)

,tagId,tag,movieId,relevance
0,1,007,1,0.02500
1,1,007,2,0.03975


In [162]:
genome_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11709768 entries, 0 to 11709767
Data columns (total 4 columns):
tagId        int64
tag          object
movieId      int64
relevance    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 357.4+ MB


In [163]:
genome_pivot_df = pd.pivot_table(genome_df, values = 'relevance' , index = 'movieId', columns = 'tag')

In [164]:
genome_pivot_df.head(3)

tag,007,007 (series),18th century,1920s,1930s,1950s,1960s,1970s,1980s,19th century,...,world politics,world war i,world war ii,writer's life,writers,writing,wuxia,wwii,zombie,zombies
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.02500,0.02500,0.05775,0.09675,0.14675,0.2170,0.06700,0.26275,0.26200,0.03200,...,0.03950,0.01800,0.04575,0.03275,0.1250,0.04150,0.01925,0.03625,0.07775,0.02300
2,0.03975,0.04375,0.03775,0.04800,0.11025,0.0725,0.04775,0.10975,0.09925,0.02050,...,0.04175,0.01925,0.01725,0.02425,0.1255,0.02250,0.01550,0.01475,0.09025,0.01875
3,0.04350,0.05475,0.02800,0.07700,0.05400,0.0685,0.05600,0.18500,0.04925,0.02675,...,0.04150,0.02675,0.02775,0.03425,0.1555,0.03675,0.01700,0.01950,0.09700,0.01850


In [165]:
# left joining the genome features to the movies dataframe
df = pd.merge(left = movies_vecrt_df, 
                right = genome_pivot_df, 
                left_on=movies_df['movieId'], 
                right_on = genome_pivot_df.index,
                how = 'left').drop('key_0', axis = 1)
df.head(3)

,movieId,title,decade_1890,decade_1900,decade_1910,decade_1920,decade_1930,decade_1940,decade_1950,decade_1960,...,world politics,world war i,world war ii,writer's life,writers,writing,wuxia,wwii,zombie,zombies
0,1,Toy Story (1995),0,0,0,0,0,0,0,0,...,0.03950,0.01800,0.04575,0.03275,0.1250,0.04150,0.01925,0.03625,0.07775,0.02300
1,2,Jumanji (1995),0,0,0,0,0,0,0,0,...,0.04175,0.01925,0.01725,0.02425,0.1255,0.02250,0.01550,0.01475,0.09025,0.01875
2,3,Grumpier Old Men (1995),0,0,0,0,0,0,0,0,...,0.04150,0.02675,0.02775,0.03425,0.1555,0.03675,0.01700,0.01950,0.09700,0.01850


In [167]:
# resetting the index to title and removing columns that are not features of the movie
df.index = df['title']
df = df.drop(columns = ['title','movieId'])
df.head()

,decade_1890,decade_1900,decade_1910,decade_1920,decade_1930,decade_1940,decade_1950,decade_1960,decade_1970,decade_1980,...,world politics,world war i,world war ii,writer's life,writers,writing,wuxia,wwii,zombie,zombies
title,,,,,,,,,,,,,,,,,,,,,
Toy Story (1995),0,0,0,0,0,0,0,0,0,0,...,0.03950,0.01800,0.04575,0.03275,0.12500,0.04150,0.01925,0.03625,0.07775,0.02300
Jumanji (1995),0,0,0,0,0,0,0,0,0,0,...,0.04175,0.01925,0.01725,0.02425,0.12550,0.02250,0.01550,0.01475,0.09025,0.01875
Grumpier Old Men (1995),0,0,0,0,0,0,0,0,0,0,...,0.04150,0.02675,0.02775,0.03425,0.15550,0.03675,0.01700,0.01950,0.09700,0.01850
Waiting to Exhale (1995),0,0,0,0,0,0,0,0,0,0,...,0.05750,0.03375,0.02275,0.03975,0.18525,0.05925,0.01500,0.01525,0.06450,0.01300
Father of the Bride Part II (1995),0,0,0,0,0,0,0,0,0,0,...,0.04250,0.02825,0.02150,0.02600,0.14275,0.02075,0.01650,0.01675,0.10750,0.01825


### Creating Recommendation Matrix

In [168]:
# transforming the data to sparse matrix
df_sparse = sparse.csr_matrix(df.fillna(0))

In [169]:
recommender = pairwise_distances(df_sparse, metric = 'cosine')

In [170]:
# Checking the shape to see the alignment of movies with other movies
recommender.shape

(27278, 27278)

In [171]:
# Creating a dataframe for the recommender
recommender_df = pd.DataFrame(recommender, index = df.index, columns = df.index)
recommender_df.head()

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Ants in the Pants (2000),Werner - Gekotzt wird später (2003),Brother Bear 2 (2006),No More School (2000),Forklift Driver Klaus: The First Day on the Job (2001),Kein Bund für's Leben (2007),"Feuer, Eis & Dosenbier (2002)",The Pirates (2014),Rentun Ruusu (2001),Innocence (2014)
title,,,,,,,,,,,,,,,,,,,,,
Toy Story (1995),0.000000,0.222372,0.317415,0.377871,0.335517,0.403743,0.364206,0.284261,0.480431,0.383067,...,0.935196,0.870393,0.770884,0.920632,0.935196,0.920632,0.920632,0.920632,1.0,0.887757
Jumanji (1995),0.222372,0.000000,0.352592,0.408555,0.344576,0.495507,0.393309,0.216954,0.382630,0.356511,...,1.000000,1.000000,0.823329,1.000000,1.000000,1.000000,1.000000,0.897999,1.0,0.855749
Grumpier Old Men (1995),0.317415,0.352592,0.000000,0.227010,0.146934,0.388472,0.200862,0.320446,0.401178,0.363687,...,0.777392,0.888696,0.921296,0.863681,0.888696,0.863681,0.863681,1.000000,1.0,1.000000
Waiting to Exhale (1995),0.377871,0.408555,0.227010,0.000000,0.264884,0.429322,0.177504,0.314381,0.446998,0.442683,...,0.775617,0.887808,0.920668,0.862594,0.887808,0.862594,0.862594,1.000000,1.0,1.000000
Father of the Bride Part II (1995),0.335517,0.344576,0.146934,0.264884,0.000000,0.483961,0.217024,0.352655,0.427506,0.421202,...,0.901292,0.901292,0.930203,0.879108,0.901292,0.879108,0.879108,1.000000,1.0,1.000000


In [172]:
# Creating a formula to search through the recommendation engine and provide the top 10 picks

def top_ten_recommendations(search):
    for title in movies_df[movies_df['title'].str.lower().str.contains(search.lower())]['title'].values:
        print(title)
#         print('Average Rating', df[df['title'] == title]['rating'].mean())
#         print('Number of Ratings', df[df['title']==title].shape[0])
        print(" ")
        print(recommender_df[title].sort_values()[1:11])
        print(" ")
        print(" ")

In [174]:
top_ten_recommendations('elf')

Twelfth Night (1996)
 
title
Much Ado About Nothing (1993)                             0.082512
Shakespeare in Love (1998)                                0.115493
Ideal Husband, An (1999)                                  0.124189
Hamlet (1996)                                             0.127558
Othello (1995)                                            0.128022
Importance of Being Earnest, The (2002)                   0.135214
William Shakespeare's Romeo + Juliet (1996)               0.137299
William Shakespeare's A Midsummer Night's Dream (1999)    0.145360
Emma (1996)                                               0.146353
Taming of the Shrew, The (1967)                           0.147844
Name: Twelfth Night (1996), dtype: float64
 
 
Me Myself I (2000)
 
title
Chaos Theory (2007)                  0.136084
Ghost Town (2008)                    0.146362
All I Want (Try Seventeen) (2002)    0.149280
Stranger than Fiction (2006)         0.149314
Family Man, The (2000)               0.1503

title
Life and Nothing But (Vie et rien d'autre, La) (1989)                  2.220446e-16
Space (1985)                                                           2.220446e-16
My Letter to George (Mesmerized) (1986)                                2.220446e-16
That Night in Varennes (Nuit de Varennes, La) (1982)                   2.220446e-16
Playing for Time (1980)                                                2.220446e-16
Rikyu (1989)                                                           2.220446e-16
Berta's Motives (Los motivos de Berta: Fantasía de Pubertad) (1985)    2.220446e-16
Speaking Parts (1989)                                                  2.220446e-16
The Diary of Anne Frank (1980)                                         2.220446e-16
Fresh Horses (1988)                                                    2.220446e-16
Name: Every Man for Himself (Slow Motion) (Sauve qui peut (la vie)) (1980), dtype: float64
 
 
Tyler Perry's I Can Do Bad All by Myself (2009)
 
title
Adr

title
I Feel Sleepy (2012)                        2.220446e-16
Corner Gas: The Movie (2014)                2.220446e-16
Klown: The Movie (Klovn) (2010)             2.220446e-16
Senotaji (2013)                             2.220446e-16
Mission London (2010)                       2.220446e-16
Vuonna 85 (2013)                            2.220446e-16
Let's Kill Ward's Wife (2014)               2.220446e-16
How to Meet Girls from a Distance (2012)    2.220446e-16
A Merry Friggin' Christmas (2014)           2.220446e-16
Few Best Men, A (2011)                      2.220446e-16
Name: Me, Myself and Mum (Les garçons et Guillaume, à table!) (2013), dtype: float64
 
 
i hate myself :) (2013)
 
title
Other F Word, The (2011)                                            0.000000
Advanced Style (2014)                                               0.000000
(A)sexual (2011)                                                    0.000000
i hate myself :) (2013)                                             0.00